Create Dateset

In [1]:
from janome.tokenizer import Tokenizer
import numpy as np

# データ読み込み
data_path = "C:\\Users\\konolab\\Desktop\\Research\\ExplainDetecter\\source_5.txt"
file = open(data_path , "r" , encoding= "UTF-8")
data = file.read()
file.close()

# 改行でデータを分割
sentences = data.splitlines()

# トークナイザを作る
tokenizer = Tokenizer()

# 単語ごとに分かれたデータを持っている
word_list =[] # List[List[str]]
print(type(sentences[0]))

# 単語ごとに分割し、そのリストをword_listに追加
for sentence in sentences:
    tokens = tokenizer.tokenize(sentence)
    tmp_words = []
    for token in tokens:
        # print(token.part_of_speech)
        tmp_words.append(token.base_form)        
    word_list.append(tmp_words)

<class 'str'>


In [2]:
dataset_path = "C:\\Users\\konolab\\Desktop\\Research\\ExplainDetecter\\learning.txt"

# datasetfile = np.load(dataset_path,delimiter = "\n",dtype = "unicode")
datasetfile = open(dataset_path , "r", encoding = "UTF-8")
dataset = datasetfile.read()
datasetfile.close()

datasets = dataset.splitlines()

datasets_nd = np.array(datasets)
labels_before = [0] * 70
labels_after = [1] * 30
labels = labels_before + labels_after

# データとラベルの結合
DATASET = []
for i in range(100):
    DATASET.append([[datasets_nd[i]],[labels[i]]])

DATASET_nd = np.array(DATASET)

# シャッフル
np.random.shuffle(DATASET_nd)


# データとラベルの分離
train_data = []
train_label = []

for j in range(100):
    train_data.append(str(DATASET_nd[j,0]))
    train_label.append(int(DATASET_nd[j,1]))
    
tokenizer_train = Tokenizer()
train_data_list = []

for data in train_data:
    tokens = tokenizer_train.tokenize(data)
    tmp_words = []
    for token in tokens:
        tmp_words.append(token.base_form)        
    train_data_list.append(tmp_words)

train_label_nd = np.array(train_label)
train_data_nd= np.array(train_data_list)


# 要素数が違うから変換できない感じ


train_label_nd =np.asarray(train_label_nd).astype(np.float32)


C:\Users\konolab\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:45: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [4]:
import tensorflow as tf
from gensim.models import Word2Vec

from keras.preprocessing.text import Tokenizer
import numpy as np
import spacy
import sys
count = 0

for i in range(100):
    if count < len(train_data_nd[i]):
        count = len(train_data_nd[i])
    train_data_nd[i] = np.array(train_data_list[0])
    
# w2v_2 = Word2Vec(train_data_list,vector_size = 64,min_count = 5, window = 5,epochs = 5)
    
tokenizer_keras2 = Tokenizer()
tokenizer_keras2.fit_on_texts(train_data_list)
train_mat = tokenizer_keras2.texts_to_matrix(train_data_list,"tfidf")
train_mat = np.resize(train_mat,100)


# tokenizer_keras2.fit_on_texts(train_data_list)
# word_index2 = tokenizer_keras2.word_index
# num_words2 = len(word_index2)
# train_data_mat = np.zeros((100,))
# for word, i in word_index2.items():
#     if word in w2v_2.wv.index_to_key:
#         train_data_mat[i] = w2v_2.wv[word]
# for i in range(100):
#     train_data_nd[i] = np.resize(train_data_nd[i],count)


In [11]:
print(train_mat.shape)

(100,)


Word2Vec

In [6]:

w2v_size = 64

# iter:機械学習の繰り返し回数（デフォルトは５）十分に学習できていないときに調整

# Word2Vecの学習　ここで
w2v = Word2Vec(word_list,vector_size = w2v_size,min_count = 5, window = 5,epochs = 5)

#  words = list(w2v.wv.index_to_key)


# word_index = {key:int(key) for key in w2v.wv.key_to_index}
# word_index['0'] = 0
# emb_mat = get_embedding_matrix(w2v , word_index)
# emb.mat.shape

tokenizer_keras = Tokenizer()
tokenizer_keras.fit_on_texts(word_list)
word_index = tokenizer_keras.word_index
num_words = len(word_index)
embedding_matrix = np.zeros((num_words+1, 64))
for word, i in word_index.items():
    if word in w2v.wv.index_to_key:
        embedding_matrix[i] = w2v.wv[word]




In [7]:
print(word_list[5])

['PC', '単体', 'で', 'COM', 'ポート', 'の', '接続', 'テスト', 'を', 'する', 'ため', 'に', '、', 'com', '0', 'com', 'を', 'インストール', 'する', '「', 'com', '0', 'com', '」', 'で', 'Google', '検索', '(.', 'jp', ')', 'する', 'と', '、', 'この', '記事', 'が', '1', '番目', 'に', 'ヒット', 'する']


In [8]:

from keras.layers import Embedding

def get_embedding_matrix(model , word_index):
    embedding_matrix = np.zeros((max(list(word_index.values())) + 1, model.vector_size),dtype = "float32")
    
    for word , labale in word_index.items():
        try:
            embedding_matrix[label] = model.wv[word]
        except KeyError:
            pass
    return embedding_matrix
    

In [9]:
num_words,w2v_size = embedding_matrix.shape

Embedding(num_words , w2v_size,
         weights=[embedding_matrix],
         input_length=128,
         trainable=False)



LSTM

In [10]:

from keras.models import Sequential
import pandas as pd
from tensorflow.keras.layers import LSTM
from keras.layers import Activation, Dense



pd.DataFrame(word_list)
max_len = max(map(len,word_list))
print(max_len)
# pd.DataFrame(word_list)

num_words, w2v_size = embedding_matrix.shape

model = Sequential()
model.add(Embedding(num_words,w2v_size,
                   weights = [embedding_matrix],
                   input_length=max_len,
                   trainable=False))
model.add(LSTM(128,input_shape=(100,),return_sequences=False))
model.add(Dense(8))
model.add(Activation("sigmoid"))

model.summary()
model.compile(loss='categorical_crossentropy',
             optimizer='sgd',
             metrics=['accuracy'])
model.fit(x = train_mat, y = train_label_nd,batch_size= 32 , epochs = 5)

#  データセット読み込んで学習してみよう！

4815
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 4815, 64)          4124800   
_________________________________________________________________
lstm (LSTM)                  (None, 128)               98816     
_________________________________________________________________
dense (Dense)                (None, 8)                 1032      
_________________________________________________________________
activation (Activation)      (None, 8)                 0         
Total params: 4,224,648
Trainable params: 99,848
Non-trainable params: 4,124,800
_________________________________________________________________
Epoch 1/5


ValueError: in user code:

    C:\Users\konolab\anaconda3\envs\tensorflow-gpu\lib\site-packages\keras\engine\training.py:853 train_function  *
        return step_function(self, iterator)
    C:\Users\konolab\anaconda3\envs\tensorflow-gpu\lib\site-packages\keras\engine\training.py:842 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\konolab\anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\konolab\anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\konolab\anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\konolab\anaconda3\env